In [9]:
Pkg.add("ColorSchemes")

  Updating registry at `/opt/julia/registries/General`
 Resolving package versions...


┌ Warning: Some registries failed to update:
│     — `/opt/julia/registries/General` — registry dirty
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Pkg/src/Types.jl:1171


  Updating `/opt/julia/environments/v1.2/Project.toml`
  [35d6a980] + ColorSchemes v3.9.0
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]


In [10]:
script_start_time = time()
println("\nimporting packages...")

import Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using JLD
using Plots
using ColorSchemes
using JSON
using Pkg
Pkg.add("StatsPlots")
Pkg.add("ColorSchemes")
using StatsPlots
using ColorSchemes

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)


importing packages...
 Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]


┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.2/StatsPlots/SiylL.ji for StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1240


packages and functions imported
Elapsed time = 16.104110956192017 seconds 
 


In [11]:
### Set parameters

# Read from config file
config_file = "config090120.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "$(homedir())/processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("bettiBars",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING,x), betti_files)
println("Located the following graph files:")
for betti_file in betti_files
    println(betti_file)
end


# Locate the nametags
nametags = []
for betti_file in betti_files
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettiBars")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)

Located the following graph files:
IID_70nodes_50reps_090120_backward_bettiBars.jld
IID_70nodes_50reps_090120_forward_bettiBars.jld
IID_70nodes_50reps_090120_thresh005_edge121_noiseOnly_bettiBars.jld
IID_70nodes_50reps_090120_thresh005_edge121_threshold_bettiBars.jld
IID_70nodes_50reps_090120_thresh005_edge121_threshold_bettiBars_postnoise.jld
IID_70nodes_50reps_090120_thresh005_edge121_threshold_bettiBars_prenoise.jld
IID_70nodes_50reps_090120_thresh015_edge363_noiseOnly_bettiBars.jld
IID_70nodes_50reps_090120_thresh015_edge363_threshold_bettiBars.jld
IID_70nodes_50reps_090120_thresh015_edge363_threshold_bettiBars_postnoise.jld
IID_70nodes_50reps_090120_thresh015_edge363_threshold_bettiBars_prenoise.jld
IID_70nodes_50reps_090120_thresh01_edge242_noiseOnly_bettiBars.jld
IID_70nodes_50reps_090120_thresh01_edge242_threshold_bettiBars.jld
IID_70nodes_50reps_090120_thresh01_edge242_threshold_bettiBars_postnoise.jld
IID_70nodes_50reps_090120_thresh01_edge242_threshold_bettiBars_prenoise.jld

cosineGeometric_70nodes_50reps_3_090120_thresh02_edge483_threshold_bettiBars_postnoise.jld
cosineGeometric_70nodes_50reps_3_090120_thresh02_edge483_threshold_bettiBars_prenoise.jld
cosineGeometric_70nodes_50reps_3_090120_thresh03_edge725_noiseOnly_bettiBars.jld
cosineGeometric_70nodes_50reps_3_090120_thresh03_edge725_threshold_bettiBars.jld
cosineGeometric_70nodes_50reps_3_090120_thresh03_edge725_threshold_bettiBars_postnoise.jld
cosineGeometric_70nodes_50reps_3_090120_thresh03_edge725_threshold_bettiBars_prenoise.jld
cosineGeometric_70nodes_50reps_3_090120_thresh04_edge966_noiseOnly_bettiBars.jld
cosineGeometric_70nodes_50reps_3_090120_thresh04_edge966_threshold_bettiBars.jld
cosineGeometric_70nodes_50reps_3_090120_thresh04_edge966_threshold_bettiBars_postnoise.jld
cosineGeometric_70nodes_50reps_3_090120_thresh04_edge966_threshold_bettiBars_prenoise.jld
cosineGeometric_70nodes_50reps_3_090120_thresh05_edge1208_noiseOnly_bettiBars.jld
cosineGeometric_70nodes_50reps_3_090120_thresh05_ed

dotProduct_70nodes_50reps_3_090120_thresh05_edge1208_threshold_bettiBars_postnoise.jld
dotProduct_70nodes_50reps_3_090120_thresh05_edge1208_threshold_bettiBars_prenoise.jld
dotProduct_70nodes_50reps_3_090120_thresh06_edge1449_noiseOnly_bettiBars.jld
dotProduct_70nodes_50reps_3_090120_thresh06_edge1449_threshold_bettiBars.jld
dotProduct_70nodes_50reps_3_090120_thresh06_edge1449_threshold_bettiBars_postnoise.jld
dotProduct_70nodes_50reps_3_090120_thresh06_edge1449_threshold_bettiBars_prenoise.jld
geometricConf_70nodes_50reps_001_100_090120_backward_bettiBars.jld
geometricConf_70nodes_50reps_001_100_090120_forward_bettiBars.jld
geometricConf_70nodes_50reps_001_100_090120_thresh005_edge121_noiseOnly_bettiBars.jld
geometricConf_70nodes_50reps_001_100_090120_thresh005_edge121_threshold_bettiBars.jld
geometricConf_70nodes_50reps_001_100_090120_thresh005_edge121_threshold_bettiBars_postnoise.jld
geometricConf_70nodes_50reps_001_100_090120_thresh005_edge121_threshold_bettiBars_prenoise.jld
geom

18-element Array{Any,1}:
 "backward"                   
 "forward"                    
 "thresh005_edge121_noiseOnly"
 "thresh005_edge121_threshold"
 "thresh015_edge363_noiseOnly"
 "thresh015_edge363_threshold"
 "thresh01_edge242_noiseOnly" 
 "thresh01_edge242_threshold" 
 "thresh02_edge483_noiseOnly" 
 "thresh02_edge483_threshold" 
 "thresh03_edge725_noiseOnly" 
 "thresh03_edge725_threshold" 
 "thresh04_edge966_noiseOnly" 
 "thresh04_edge966_threshold" 
 "thresh05_edge1208_noiseOnly"
 "thresh05_edge1208_threshold"
 "thresh06_edge1449_noiseOnly"
 "thresh06_edge1449_threshold"

In [12]:
for nametag in nametags
    
    ## Create one large plot with all of the betti curves
    betti_files_nametag = filter(x -> occursin("$(nametag)",x), betti_files)
    model_names = [split(betti_file_nametag,"_")[1] for betti_file_nametag in betti_files_nametag]
  

    ### Create one large plot with all of the betti curves
    nModels = length(betti_files_nametag)
    nEdges = binomial(NNODES, 2)
    bettiBarAll = zeros(NREPS, MAXDIM, nModels)
    muBarAll = zeros(NREPS, MAXDIM, nModels)
    nuBarAll = zeros(NREPS, MAXDIM, nModels)
    
    for (i,betti_file_nametag) in enumerate(betti_files_nametag)

        # Read in Betti curves
        bettiBar_dict = load("$(read_dir)/$(betti_file_nametag)")
        bettiBarAll[:,:, i] = bettiBar_dict["bettiBarArray"]
        muBarAll[:,:, i] = bettiBar_dict["muBarArray"]
        nuBarAll[:,:, i] = bettiBar_dict["nuBarArray"]

    end


    println("$(nametag) Size bettiBarAll is $(size(bettiBarAll))")



    # Plot
    p1a = violin(bettiBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p1b = violin(bettiBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p1c = violin(bettiBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("BettiBar")

    plot(p1a, p1b, p1c, layout = (3,1))

    savefig("../figures/all_bettiBars_$(nametag).pdf")


    p2a = violin(muBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p2b = violin(muBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p2c = violin(muBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("muBar")

    plot(p2a, p2b, p2c, layout = (3,1))

    savefig("../figures/all_muBars_$(nametag).pdf")


    p3a = violin(nuBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p3b = violin(nuBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p3c = violin(nuBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("muBar")

    plot(p3a, p3b, p3c, layout = (3,1))

    savefig("$(homedir())/figures/all_nuBars_$(nametag).pdf")

end

backward Size bettiBarAll is (50, 3, 10)
forward Size bettiBarAll is (50, 3, 10)
thresh005_edge121_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh005_edge121_threshold Size bettiBarAll is (50, 3, 30)
thresh015_edge363_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh015_edge363_threshold Size bettiBarAll is (50, 3, 30)
thresh01_edge242_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh01_edge242_threshold Size bettiBarAll is (50, 3, 30)
thresh02_edge483_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh02_edge483_threshold Size bettiBarAll is (50, 3, 30)
thresh03_edge725_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh03_edge725_threshold Size bettiBarAll is (50, 3, 30)
thresh04_edge966_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh04_edge966_threshold Size bettiBarAll is (50, 3, 30)
thresh05_edge1208_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh05_edge1208_threshold Size bettiBarAll is (50, 3, 30)
thresh06_edge1449_noiseOnly Size bettiBarAll is (50, 3, 10)
thresh06_edge1449_threshold